In [7]:
import pymupdf
import langextract as lx
import textwrap


In [ ]:
# Run this cell once

!curl https://arxiv.org/pdf/1704.05842 -o jet_substructure_paper.pdf

In [ ]:
doc = pymupdf.open('jet_substructure_paper.pdf')

doc

In [ ]:
for page in doc:
    text = page.get_text()
    print(text)

In [ ]:
prompt = textwrap.dedent("""\
    Look for mentions of  the specific dataset, mentions of number of events 
    used and/or disk/file size.""")

In [ ]:
example1 = [
    lx.data.ExampleData(
        text = (
            "There are 1664 AOD files in the Jet Primary Dataset, corresponding to 20,022,826 events"
            "and 2.0 Terabytes of disk space. Within CMSSW, it is possible to access the AOD files remotely through the"
            "XRootD interface [218]. We found it more convenient to first download the AOD files and then process them locally," 
            "being careful to maintain the same directory structure as on the Open Data servers in order to ensure consistency of the workflow."
        ),
        extractions = [
            lx.data.Extraction(
                extraction_class = "Amount of data used, the specific dataset, and number of events",
                extraction_text = "1664 AOD files in the Jet Primary Dataset, corresponding to 20,022,826 events and 2.0 Terabytes of disk space",
                attributes = {
                    "Dataset": "Jet Primary Dataset",
                    "Number of files": "1664",
                    "Number of events": "20,022,826",
                    "Disk size": "2.0 Terabytes"
                },
                
            )
        ]
    )
]

example2 = [
    lx.data.ExampleData(
        text = (
            "The data used in this analysis corresponds to an integrated luminosity of 2.3 fb-1 of proton-proton collisions at √s = 13 TeV recorded by the CMS experiment"
            "in 2015. The data were collected using a trigger that requires at least one jet with pT > 450 GeV and |η| < 3.0. After applying data quality criteria,"
            "the total number of events passing the trigger is approximately 1.2 million, corresponding to a total disk size of about 150 GB."
        ),
        extractions = [
            lx.data.Extraction(
                extraction_class = "Amount of data used, the specific dataset, and number of events",
                extraction_text = "the total number of events passing the trigger is approximately 1.2 million, corresponding to a total disk size of about 150 GB",
                attributes = {
                    "Number of events": "approximately 1.2 million",
                    "Experiment": "CMS",
                    "Disk size": "150 GB"
                    "Date": "2015"
                },
                
            ),      
            
        ]
    
    )
]

#example3 = []